In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(3)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Training

In [5]:
name = 'baseline_full_effb3_alb'

In [6]:
import geffnet
m = geffnet.efficientnet_b3(pretrained=True, drop_rate=0.25, drop_connect_rate=0.2)
m.classifier = nn.Linear(m.classifier.in_features, 6)

In [7]:
dbch = get_album_data(512, 128, splits=Meta.splits_stg1)
learn = get_learner(dbch, m)

## Training

In [8]:
# do_fit(learn, 12, 4e-2)
# learn.save(f'runs/{name}-1')

In [9]:
# learn.dls = get_album_data(128, 256, splits=Meta.splits_stg1)
# do_fit(learn, 12, 4e-3, freeze=False)
# learn.save(f'runs/{name}-2')

In [10]:
learn.dls = get_album_data(64, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 4e-4, freeze=False)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.080911,0.086344,0.972935,0.946604,2:07:54
1,0.072699,0.075303,0.976742,0.954841,2:08:10
2,0.065038,0.071434,0.978074,0.957413,2:06:33
3,0.056279,0.072142,0.978233,0.957948,2:06:55


In [8]:
learn.load(f'runs/{name}-3')
learn.dls = get_album_data(128, 256, splits=Meta.splits_stg1)
do_fit(learn, 4, 1e-3, freeze=False)
learn.save(f'runs/{name}-3-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.080477,0.084183,0.974541,0.949443,58:31
1,0.072738,0.079129,0.975761,0.953479,58:37
2,0.064752,0.077867,0.976793,0.954650,59:20
3,0.053222,0.079663,0.976992,0.955847,1:00:03


In [9]:
learn.dls = get_album_data(64, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 2, 1e-4, freeze=False)
learn.save(f'runs/{name}-3-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.052700,0.076695,0.977436,0.956382,2:07:56
1,0.049106,0.077631,0.977720,0.957056,2:07:54


## Submission

In [7]:
learn.load(f'runs/{name}-3')

In [8]:
# tst_fns = df_tst.index.values

In [9]:
sub_fn = f'subm/{name}-3'

In [15]:
s1 = L(range(len(Meta.df_tst.index)))
learn.dls = get_album_data_gen(Meta.df_tst.index, bs=512, img_tfm=get_cv2_fn(path/'tst_jpg'), sz=384, splits=[s1, s1], test=True)

In [16]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dls = tst_dbch

In [17]:
preds,targs = learn.get_preds()

In [18]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [19]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1_album-3.csv

In [20]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.078346', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 7.41MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [25]:
# using albumentation tfms
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14277854,
 'totalBytes': 27277209,
 'date': '2020-01-24T01:13:44.25Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.087925',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1_album-3.csv',
 'publicScore': '0.93388',
 'privateScore': '0.06417',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14277854/14277854.raw'}

In [21]:
# This was with old data loader (fastai's not albumentation tfms)
# # learn.dls = get_test_data(Meta.df_tst, bs=512, sz=384) # old
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14266098,
 'totalBytes': 27277209,
 'date': '2020-01-22T18:41:10.043Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.087925',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1_album-3.csv',
 'publicScore': '0.52899',
 'privateScore': '0.06902',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14266098/14266098.raw'}